In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

# Riemann sheets for two channels

In [ ]:
%pip install -q ampform==0.14.8 plotly==5.17.0 sympy==1.12

In [ ]:
from __future__ import annotations

import warnings
from typing import Any

import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import sympy as sp
from ampform.io import aslatex
from ampform.sympy import unevaluated
from IPython.display import Math

warnings.filterwarnings("ignore")

## Squareroot definition 
See report on Riemann sheets for one channel case [Link]

In [ ]:
@unevaluated(real=False)
class SignedSqrt(sp.Expr):
    z: Any
    _latex_repr_ = R"\sqrt[+]{{{z}}}"

    def evaluate(self) -> sp.Expr:
        z = self.args[0]
        return sp.sqrt(abs(z)) * sp.exp(sp.I * PosArg(z) / 2)


@unevaluated
class PosArg(sp.Expr):
    z: Any
    _latex_repr_ = R"\arg^+\left({z}\right)"

    def evaluate(self) -> sp.Expr:
        z = self.args[0]
        arg = sp.arg(z)
        return sp.Piecewise(
            (arg - 2 * sp.pi, arg < -2 * sp.pi),
            (arg + 2 * sp.pi, arg > 0),
            (arg, True),
        )


z = sp.Symbol("z", complex=True)
Math(aslatex({e: e.evaluate() for e in [SignedSqrt(z), PosArg(z)]}))

In [ ]:
@unevaluated(real=False)
class PhaseSpaceFactor(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho^+\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return SignedSqrt((s - ((m1 + m2) ** 2)) * (s - (m1 - m2) ** 2) / s**2)


s, m_a, m_b = sp.symbols("s m_a m_b")
rho_expr = PhaseSpaceFactor(s, m_a, m_b)
Math(aslatex({rho_expr: rho_expr.doit(deep=False)}))

In [ ]:
@unevaluated(real=False)
class BreakupMomentum(sp.Expr):
    s: Any
    m_a: Any
    m_b: Any
    _latex_repr_ = R"q^+\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m_a, m_b = self.args
        return SignedSqrt((s - (m_a + m_b) ** 2) * (s - (m_a - m_b) ** 2) / (s * 4))


breakup_momentum = BreakupMomentum(s, m_a, m_b)
Math(aslatex({breakup_momentum: breakup_momentum.doit(deep=False)}))

In [ ]:
@unevaluated(real=False)
class ChewMandelstamm(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho^\text{{CM}}\left({s},{m1},{m2}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        q_break = BreakupMomentum(s, m1, m2)
        return (
            1
            / (16 * (sp.pi) ** 2)
            * (
                (2 * q_break / sp.sqrt(s))
                * sp.log(
                    (m1**2 + m2**2 - s + 2 * sp.sqrt(s) * q_break) / (2 * m1 * m2)
                )
                - (m1**2 - m2**2) * (1 / s - 1 / (m1 + m2) ** 2) * sp.log(m1 / m2)
            )
        )


cm_expr = ChewMandelstamm(s, m_a, m_b)
Math(aslatex({cm_expr: cm_expr.doit(deep=False)}))

In [ ]:
@unevaluated(real=False)
class PhaseSpaceFactor_breakup(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho^+\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return SignedSqrt((s - ((m1 + m2) ** 2)) * (s - (m1 - m2) ** 2) / s**2)


s, m_a, m_b = sp.symbols("s m_a m_b")
rho_expr = PhaseSpaceFactor(s, m_a, m_b)
Math(aslatex({rho_expr: rho_expr.doit(deep=False)}))

### Visual comparison

In [ ]:
args = (s, m_a, m_b)
cm_func_sym = sp.lambdify(args, cm_expr.doit())
rho_func = sp.lambdify(args, rho_expr.doit())

s_values = np.linspace(0, 2, 400)
m1_val = 0.6
m2_val = 0.4
arg_vals = (m1_val, m2_val)

cm_values = 16 * np.pi * cm_func_sym(s_values, *arg_vals)
rho_values = 1j * rho_func(s_values, *arg_vals)

fig, axes = plt.subplots(figsize=(15, 6), ncols=2, sharex=True, sharey=True)
ax1, ax2 = axes
ax1.plot(s_values, cm_values.real, label="Real part")
ax1.plot(s_values, cm_values.imag, label="Imaginary part")
ax1.set_xlabel("Re(s)")
ax1.set_ylabel("Function value")
ax1.set_title("Chew Mandelstamm Function")
ax1.legend()

ax2.plot(s_values, rho_values.real, label="Real part")
ax2.plot(s_values, rho_values.imag, label="Imaginary part")
ax2.set_xlabel("Re(s)")
ax2.set_title("Phase Space Factor")
ax2.legend()


ax2.set_ylim(-1.2, +1.6)
plt.tight_layout()

plt.show()

## Define parameters values for plotting 

## T matrix definition with K matrix

In [ ]:
n_channels = 2
I = sp.Identity(n_channels)
K = sp.MatrixSymbol("K", n_channels, n_channels)
rho_symbol = sp.MatrixSymbol("rho", n_channels, n_channels)
rho = sp.DiagonalMatrix(rho_symbol)
rho.as_explicit()

In [ ]:
T = (I - sp.I * K * rho).inv() * K
T

In [ ]:
T_explicit = T.as_explicit()
T_explicit

No Blatt-Weisskopf-Formfactors because $L=0$:

In [ ]:
s = sp.Symbol("s")
m_a_1 = sp.Symbol(R"m_{a,1}")
m_b_1 = sp.Symbol(R"m_{b,1}")
m_a_2 = sp.Symbol(R"m_{a,2}")
m_b_2 = sp.Symbol(R"m_{b,2}")
w_R = sp.Symbol(R"\Gamma_{R}")
gamma_R_1 = sp.Symbol(R"\gamma_{R,1}")
gamma_R_2 = sp.Symbol(R"\gamma_{R,2}")
m_R = sp.Symbol(R"m_R")

k_expr_00 = (gamma_R_1 * gamma_R_1 * m_R * w_R) / (s - m_R**2)
k_expr_10 = (gamma_R_1 * gamma_R_2 * m_R * w_R) / (s - m_R**2)
k_expr_11 = (gamma_R_2 * gamma_R_2 * m_R * w_R) / (s - m_R**2)

In [ ]:
rho_expressions = {
    K[0, 0]: k_expr_00,
    K[1, 1]: k_expr_11,
    K[0, 1]: k_expr_10,
    K[1, 0]: k_expr_10,
    rho[0, 0]: PhaseSpaceFactor(s, m_a_1, m_b_1),
    rho[1, 1]: PhaseSpaceFactor(s, m_a_2, m_b_2),
}
Math(aslatex(rho_expressions))

In [ ]:
cm_expressions = {
    K[0, 0]: k_expr_00,
    K[1, 1]: k_expr_11,
    K[0, 1]: k_expr_10,
    K[1, 0]: k_expr_10,
    rho[0, 0]: sp.I * 16 * sp.pi * ChewMandelstamm(s, m_a_1, m_b_1),
    rho[1, 1]: sp.I * 16 * sp.pi * ChewMandelstamm(s, m_a_2, m_b_2),
}
Math(aslatex(cm_expressions))

In [ ]:
cm_conj_conj_expressions = {
    K[0, 0]: k_expr_00,
    K[1, 1]: k_expr_11,
    K[0, 1]: k_expr_10,
    K[1, 0]: k_expr_10,
    rho[0, 0]: sp.I * 16 * sp.pi * ChewMandelstamm(s, m_a_1, m_b_1).conjugate(),
    rho[1, 1]: sp.I * 16 * sp.pi * ChewMandelstamm(s, m_a_2, m_b_2).conjugate(),
}

In [ ]:
cm_conj1_expressions = {
    K[0, 0]: k_expr_00,
    K[1, 1]: k_expr_11,
    K[0, 1]: k_expr_10,
    K[1, 0]: k_expr_10,
    rho[0, 0]: sp.I * 16 * sp.pi * ChewMandelstamm(s, m_a_1, m_b_1).conjugate(),
    rho[1, 1]: sp.I * 16 * sp.pi * ChewMandelstamm(s, m_a_2, m_b_2),
}

In [ ]:
cm_conj2_expressions = {
    K[0, 0]: k_expr_00,
    K[1, 1]: k_expr_11,
    K[0, 1]: k_expr_10,
    K[1, 0]: k_expr_10,
    rho[0, 0]: sp.I * 16 * sp.pi * ChewMandelstamm(s, m_a_1, m_b_1),
    rho[1, 1]: sp.I * 16 * sp.pi * ChewMandelstamm(s, m_a_2, m_b_2).conjugate(),
}

In [ ]:
T_cm_expr_00 = T_explicit[0, 0].xreplace(cm_expressions)
T_cm_conj_conj_expr_00 = T_explicit[0, 0].xreplace(cm_conj_conj_expressions)
T_cm_conj1_expr_00 = T_explicit[0, 0].xreplace(cm_conj1_expressions)
T_cm_conj2_expr_00 = T_explicit[0, 0].xreplace(cm_conj2_expressions)

In [ ]:
T_cm_expr_00.simplify(doit=False)

## T Matrix

In [ ]:
args = (s, m_a_1, m_b_1, m_a_2, m_b_2, m_R, w_R, gamma_R_1, gamma_R_2)
T_cm_func_00 = sp.lambdify(args, T_cm_expr_00.doit())
T_cm_conj_conj_func_00 = sp.lambdify(args, T_cm_conj_conj_expr_00.doit())
T_cm_conj1_func_00 = sp.lambdify(args, T_cm_conj1_expr_00.doit())
T_cm_conj2_func_00 = sp.lambdify(args, T_cm_conj2_expr_00.doit())

In [ ]:
epsilon = 1e-5
x = np.linspace(0, 3, num=200)
y = np.linspace(epsilon, 6, num=200)
X, Yn = np.meshgrid(x, +y)
_, Yp = np.meshgrid(x, -y)
Zn = X - 0.001 * 1j
Zp = X + 0.001 * 1j

m_a_1_val = 0.14
m_b_1_val = 0.14

m_a_2_val = 0.5
m_b_2_val = 0.5

m_R = 1.0
w_R = 1.0
gamma_R_1 = 1
gamma_R_2 = 1

arg_vals = (
    m_a_1_val,
    m_b_1_val,
    m_a_2_val,
    m_b_2_val,
    w_R,
    m_R,
    gamma_R_1,
    gamma_R_2,
)

T_cm_00_p = T_cm_func_00(Zp**2, *arg_vals)
T_cm_00_n = T_cm_func_00(Zn**2, *arg_vals)

T_cm_conj_conj_00_p = T_cm_conj_conj_func_00(Zp**2, *arg_vals)
T_cm_conj_conj_00_n = T_cm_conj_conj_func_00(Zn**2, *arg_vals)

T_cm_conj1_00_p = T_cm_conj1_func_00(Zp**2, *arg_vals)
T_cm_conj1_00_n = T_cm_conj1_func_00(Zn**2, *arg_vals)

T_cm_conj2_00_p = T_cm_conj2_func_00(Zp**2, *arg_vals)
T_cm_conj2_00_n = T_cm_conj2_func_00(Zn**2, *arg_vals)

In [ ]:
plt.plot(x, T_cm_conj_conj_00_p[0].imag, c="blue", zorder=99)
plt.plot(x, T_cm_00_p[0].imag, c="yellow", zorder=99)
plt.plot(x, T_cm_conj1_00_p[0].imag, c="red", zorder=99)
plt.plot(x, T_cm_conj2_00_n[0].imag, c="green", zorder=99)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

Sn_cm = go.Surface(x=X, y=Yn, z=project(Tn_cm), **sty("Unphysical"))
Sp_cm = go.Surface(x=X, y=Yp, z=project(Tp_cm), **sty("Unphysical"))

Sn_cm_conj = go.Surface(x=X, y=Yn, z=project(Tn_cm_conj), **sty("Physical"))
Sp_cm_conj = go.Surface(x=X, y=Yp, z=project(Tp_cm_conj), **sty("Physical"))

intersection_points = []
for i in range(len(x)):
    intersection_points.append((x[i], 0, project(Tn_cm[0])[i]))

intersection_points = np.array(intersection_points)
intersection_line = go.Scatter3d(
    x=intersection_points[:, 0],
    y=intersection_points[:, 1],
    z=intersection_points[:, 2],
    mode="lines+markers",
    line=dict(color="yellow", width=20),
    marker=dict(size=1, color="pink"),
    name="Intersection Line",
)

fig = make_subplots(
    rows=1,
    cols=3,
    specs=[[{"type": "surface"}, {"type": "surface"}, {"type": "scatter3d"}]],
)

fig.add_trace(Sn_cm, row=1, col=1)
fig.add_trace(Sp_cm, row=1, col=1)

fig.add_trace(Sn_cm_conj, row=1, col=3)
fig.add_trace(Sp_cm_conj, row=1, col=3)

fig.add_trace(Sn_cm, row=1, col=2)
fig.add_trace(Sp_cm_conj, row=1, col=2)

fig.add_trace(intersection_line, row=1, col=2)

fig.update_layout(
    height=550,
    width=1200,
    showlegend=False,
    title_text=f"{projection_text} part of T function with Chew-Mandelstam",
)
fig.update_scenes(
    xaxis_title_text="Re s",
    yaxis_title_text="Im s",
    zaxis_range=[-vmax, +vmax],
)
fig.show()